In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from sklearn import svm, datasets
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import time

In [ ]:
# Iyer data import
iyer_datapath = '/mnt/c/Users/isaac/Desktop/CSE447/final_project/DataMiningProject/data/6_cho_iyer/'
iyer_filename = 'iyer.txt'

iyer_data = np.genfromtxt(iyer_datapath+iyer_filename)
iyer_X = iyer_data [:,2:]
iyer_y = iyer_data[:,1]
iyer_Xtrain, iyer_Xtest, iyer_ytrain, iyer_ytest = model_selection.train_test_split(iyer_X, iyer_y, train_size=0.80, test_size=0.20, random_state=101)
